In [1]:
import re
import pandas as pd
from tqdm import tqdm

### 1.Finding the unique words

In [2]:
words = []
with open('big.txt','r') as fd:
    lines = fd.readlines()
    
    for line in lines:
        words += re.findall(r'\w+',line.lower())
print(len(words))
vocab = list(set(words))
print(len(vocab))

1115585
32198


### 2.finding the probability distribution

In [3]:
word_probability = {}

for word in tqdm(vocab):
    word_probability[word] = float(words.count(word)/len(words))

100%|███████████████████████████████████████████████████████████████████████████████████| 32198/32198 [09:23<00:00, 57.16it/s]


### 3. Text preprocessing
splitting

In [4]:
def split(word):
    parts=[]
    
    for i in range (len(word)+1):
        parts+=[(word[:i],word[i:])]
    
    return parts

split('vasudev')

[('', 'vasudev'),
 ('v', 'asudev'),
 ('va', 'sudev'),
 ('vas', 'udev'),
 ('vasu', 'dev'),
 ('vasud', 'ev'),
 ('vasude', 'v'),
 ('vasudev', '')]

3.1 Delete


In [5]:
def delete(word):
    output = []
    for l,r in split(word):
        output.append(l+r[1:])
    return output

delete('heallo')

['eallo', 'hallo', 'hello', 'healo', 'healo', 'heall', 'heallo']

#3.2) swap

In [6]:
def swap(word):
        
    output = []    
    for l,r in split(word):
        if (len(r) > 1):
            output.append(l + r[1] + r[0] + r[2:])
    return output
            
swap('lvoe')

['vloe', 'love', 'lveo']

#3.3)replace

In [7]:
def replace(word):
    
    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []    

    for l,r in split(word):
        for char in characters:
            output.append(l + char +  r[1:])
    return output

len(replace('lave'))

130

#3.4)Insert

In [8]:
def insert(word):
    
    characters = 'abcdefghijklmnopqrstuvwxyz'
    output=[]
        
    for l,r in split(word):
        for char in characters:
            output.append(l+char+r)

        return output

insert('lve')

['alve',
 'blve',
 'clve',
 'dlve',
 'elve',
 'flve',
 'glve',
 'hlve',
 'ilve',
 'jlve',
 'klve',
 'llve',
 'mlve',
 'nlve',
 'olve',
 'plve',
 'qlve',
 'rlve',
 'slve',
 'tlve',
 'ulve',
 'vlve',
 'wlve',
 'xlve',
 'ylve',
 'zlve']

## 4. Finding the prediction
### 4.1) combining possible words

In [13]:
def edit(word):
    return set(insert(word)+delete(word)+swap(word)+replace(word))

In [10]:
def edit(word):
    return (insert(word)+ delete(word)+swap(word)+replace(word))


## 4.2) Predicting the word

In [17]:
def spell_check_edit_1(word, count = 5):
    
    output = []
    suggested_words = edit(word)
    
    for wrd in suggested_words:        
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
            
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)

In [22]:
spell_check_edit_1('famili')

['family']

In [23]:
spell_check_edit_1('tha')

['the', 'that', 'than', 'tea', 'ha']

In [24]:
spell_check_edit_1('im')

['in', 'it', 'is', 'i', 'him']

## 5. Finding the Prediction (Level - 2)

In [39]:
def spell_check_edit_2(word, count=5):
    output = []
    suggested_words = set(edit(word))  # Level one edit

    for e1 in edit(word):
        suggested_words.update(edit(e1))  # Second level edit

    for wrd in suggested_words:
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])

    return list(
        pd.DataFrame(output, columns=['word', 'prob'])
          .sort_values(by='prob', ascending=False)
          .head(count)['word']
          .values
    )
spell_check_edit_2('fameli')

['family', 'namely', 'fame', 'amelie', 'camel']

In [40]:
spell_check_edit_2('thhat')

['that', 'what', 'than', 'hat', 'heat']